# AD9833 ToolBox test with FTDI
https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [ ]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [ ]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [ ]:
paths = [['..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [ ]:
%pylab inline 

from signal_generators import adapters, tools
from signal_generators.ad983x.ad9833 import AD9833
from signal_generators.shift_register import ShiftRegister

## Debug mode?

In [ ]:
adapters.SPI.DEBUG_MODE = False  # whether to show SPI written data.
AD9833.DEBUG_MODE = False        # whether to dump registers. 

## Generators

In [ ]:
with_hardware_device = True

if with_hardware_device:
    _clk = adapters.Pin.get_Ftdi_pin(pin_id = 4)
    _data = adapters.Pin.get_Ftdi_pin(pin_id = 1)

    _ss1 = adapters.Pin.get_Ftdi_pin(pin_id = 3)
    _ss2 = adapters.Pin.get_Ftdi_pin(pin_id = 0)

    _spi_1 = ShiftRegister(stb_pin = _ss1, clk_pin = _clk, data_pin = _data, polarity = 1)
    _spi_2 = ShiftRegister(stb_pin = _ss2, clk_pin = _clk, data_pin = _data, polarity = 1)
else:
    _spi_1 = _spi_2 = _ss1 = _ss2 = None  # using None for testing without actual hardware device.

ad1 = AD9833(_spi_1, _ss1)
ad2 = AD9833(_spi_2, _ss2)

In [ ]:
ad1.reset()
ad2.reset()

In [ ]:
ad1.enable_output(False)
ad2.enable_output(False)

In [ ]:
tb = tools.ToolBox()

## Frequency Sweeping

In [ ]:
tb.sweep(ad1, freq_start = 10, freq_end = int(1e6), n_freqs = 100,
         sweep_type = 'logarithm', direction = 'round_trip', n_cycles = 2,
         slot_duration = 0.01, between_cycle_seconds = 0)

## Function Toggling

In [ ]:
duration_seconds, count, cycle_time = tb.toggle(ad1,
                                                fun = 'enable_output', params = ({'value': True}, {'value': False}),
                                                n_cycles = 10, slot_duration = 0.2, between_cycle_seconds = 0.2)
print('duration_seconds, count, cycle_time', duration_seconds, count, cycle_time)

## Waveform Juggling

In [ ]:
tb.juggle((ad1, ad2),
          freq_start = int(1e2), freq_end = int(2e4), n_freqs = 100, freqs_type = 'logarithm',
          slot_duration = 0.2, between_cycle_seconds = 0.2,
          n_juggles = 10)